# Search Planet Basemap API
This script searches the basemap API for basemap grids across the circumpolar region.

## API and Package Set-Up

In [1]:
import os
import json
import requests
import urllib.request
import re
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely as shp
from pprint import pprint

In [2]:
# Get Planet API Key
%load_ext dotenv
%dotenv

api_key = os.getenv('PL_BM_API_KEY')

## Search for Basemap

In [3]:
#setup Planet base URL
API_URL = "https://api.planet.com/basemaps/v1/mosaics"

#setup session
session = requests.Session()

#authenticate
session.auth = (api_key, "")

In [4]:
# make get request to access mosaic from basemaps API
res = session.get(API_URL)

#response status code - should be 200 if there were no errors
print(res.status_code)

#print metadata for mosaic
mosaic = res.json()
# print(json.dumps(mosaic, indent=2))


200


In [16]:
#make get request to access mosaic from basemaps API
res = session.get("https://api.planet.com/basemaps/v1/series")

#response status code
print(res.status_code)

#print names of mosaic options
series = res.json()
names = [item['name'] for item in series['series']]
pprint(names)

# print names of global Q3 (July-Sept) mosaics
mosaic_link = [
    mosaic['_links']['mosaics'] for mosaic in series['series'] if mosaic['name'] == 'Global Quarterly'
][0]
pprint(mosaic_link)
mosaics = session.get(mosaic_link).json()
mosaic_names = [mosaic['name'] for mosaic in mosaics['mosaics'] if re.match('^.*q3.*$', mosaic['name'])]
mosaic_names

200
['Global Monthly',
 'Global Quarterly',
 'PS analytic_sr biweekly subscription',
 'PS analytic_sr daily subscription',
 'PS analytic_sr monthly subscription',
 'PS analytic_sr quarterly subscription',
 'PS analytic_sr twice_monthly subscription',
 'PS analytic_sr weekly subscription',
 'PS normalized_analytic biweekly subscription',
 'PS normalized_analytic daily subscription',
 'PS normalized_analytic monthly L16 subscription',
 'PS normalized_analytic monthly subscription',
 'PS normalized_analytic quarterly subscription',
 'PS normalized_analytic twice_monthly subscription',
 'PS normalized_analytic weekly subscription',
 'PS sen2_normalized_analytic monthly subscription',
 'PS sen2_normalized_analytic quarterly subscription',
 'PS sen2_normalized_analytic_8b_sr monthly subscription',
 'PS visual biweekly subscription',
 'PS visual daily subscription',
 'PS visual twice_monthly subscription',
 'PS visual weekly subscription']
'https://api.planet.com/basemaps/v1/series/060c47b7-5

['global_quarterly_2016q3_mosaic',
 'global_quarterly_2017q3_mosaic',
 'global_quarterly_2018q3_mosaic',
 'global_quarterly_2019q3_mosaic',
 'global_quarterly_2020q3_mosaic',
 'global_quarterly_2021q3_mosaic',
 'global_quarterly_2022q3_mosaic',
 'global_quarterly_2023q3_mosaic']

In [17]:
#set params for search using name of mosaic
parameters = {
    "name__is": mosaic_names[2]
}

#make get request to access mosaic from basemaps API
res = session.get(API_URL, params = parameters)

#response status code
print(res.status_code)

#print metadata for mosaic
mosaic = res.json()
print(json.dumps(mosaic, indent=2))

200
{
  "_links": {
    "_self": "https://api.planet.com/basemaps/v1/mosaics?api_key=PLAKd5b56c4d31f8444387b2eebad24e1c02"
  },
  "mosaics": [
    {
      "_links": {
        "_self": "https://api.planet.com/basemaps/v1/mosaics/b139f491-e6ef-4106-92e8-8a8810066b85?api_key=PLAKd5b56c4d31f8444387b2eebad24e1c02",
        "quads": "https://api.planet.com/basemaps/v1/mosaics/b139f491-e6ef-4106-92e8-8a8810066b85/quads?api_key=PLAKd5b56c4d31f8444387b2eebad24e1c02&bbox={lx},{ly},{ux},{uy}",
        "tiles": "https://tiles.planet.com/basemaps/v1/planet-tiles/global_quarterly_2018q3_mosaic/gmap/{z}/{x}/{y}.png?api_key=PLAKd5b56c4d31f8444387b2eebad24e1c02"
      },
      "bbox": [
        -180,
        -56.072035,
        180,
        76.016094
      ],
      "coordinate_system": "EPSG:3857",
      "datatype": "byte",
      "first_acquired": "2018-07-01T00:00:00.000Z",
      "grid": {
        "quad_size": 4096,
        "resolution": 4.777314267823516
      },
      "id": "b139f491-e6ef-4106-92e8-

In [20]:
#get id
mosaic_id = mosaic['mosaics'][0]['id']

#get bbox for entire mosaic
mosaic_bbox = mosaic['mosaics'][0]['bbox']

print(mosaic_id)
print(mosaic_bbox)

b139f491-e6ef-4106-92e8-8a8810066b85
[-180, -56.072035, 180, 76.016094]


In [21]:
#converting bbox to string for search params
string_bbox = '-180,40,180,85'

#search for mosaic quad using AOI
search_parameters = {
    'bbox': string_bbox,
    'minimal': True
}

In [23]:
#accessing quads using metadata from mosaic
quads_url = "{}/{}/quads".format(API_URL, mosaic_id)

#send request
res = session.get(quads_url, params=search_parameters, stream=True)

quads = res.json()
# pprint(quads)

item_id = [item['id'] for item in quads['items']]
link = [item['_links']['download'] for item in quads['items']]
percent_covered = [item['percent_covered'] for item in quads['items']]
geom = [
            shp.geometry.box(item['bbox'][0], item['bbox'][1], item['bbox'][2], item['bbox'][3]) 
            for item 
            in quads['items']
        ]
# print(item_id)

try:
    next_link = quads['_links']['_next']
    # print(next_link)
except:
    next_link = None
    
while next_link:
    res = session.get(next_link, stream=True)
    quads = res.json()

    item_id = item_id + [item['id'] for item in quads['items']]
    link = link + [item['_links']['download'] for item in quads['items']]
    percent_covered = percent_covered + [item['percent_covered'] for item in quads['items']]
    geom = geom + [
        shp.geometry.box(item['bbox'][0], item['bbox'][1], item['bbox'][2], item['bbox'][3]) 
        for item 
        in quads['items']
    ]

    try:
        next_link = quads['_links']['_next']
        # print(next_link)
    except:
        next_link = None

In [24]:
yg_grids = gpd.GeoDataFrame(
    data = pd.DataFrame({
        'id': item_id,
        'link': link,
        'percent_covered': percent_covered,
    }),
    geometry = gpd.GeoSeries(
        data = geom,
        crs = 'EPSG:4326'
    )
)

yg_grids

,id,link,percent_covered,geometry
0,319-1707,https://api.planet.com/basemaps/v1/mosaics/b13...,None,"POLYGON ((-123.75000 75.97355, -123.75000 76.0..."
1,320-1707,https://api.planet.com/basemaps/v1/mosaics/b13...,None,"POLYGON ((-123.57422 75.97355, -123.57422 76.0..."
2,321-1707,https://api.planet.com/basemaps/v1/mosaics/b13...,None,"POLYGON ((-123.39844 75.97355, -123.39844 76.0..."
3,322-1707,https://api.planet.com/basemaps/v1/mosaics/b13...,None,"POLYGON ((-123.22266 75.97355, -123.22266 76.0..."
4,323-1707,https://api.planet.com/basemaps/v1/mosaics/b13...,None,"POLYGON ((-123.04688 75.97355, -123.04688 76.0..."
...,...,...,...,...
545015,1830-1272,https://api.planet.com/basemaps/v1/mosaics/b13...,None,"POLYGON ((141.85547 39.90974, 141.85547 40.044..."
545016,1831-1272,https://api.planet.com/basemaps/v1/mosaics/b13...,None,"POLYGON ((142.03125 39.90974, 142.03125 40.044..."
545017,1832-1272,https://api.planet.com/basemaps/v1/mosaics/b13...,None,"POLYGON ((142.20703 39.90974, 142.20703 40.044..."
545018,1833-1272,https://api.planet.com/basemaps/v1/mosaics/b13...,None,"POLYGON ((142.38281 39.90974, 142.38281 40.044..."


In [31]:
yg_grids.to_file('../data/yg_global_basemap_grids.geojson')